In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pinxau1000/radioml2018")

print("Path to dataset files:", path)

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\USER\.cache\kagglehub\datasets\pinxau1000\radioml2018\versions\2


In [11]:
!dir C:\Users\USER\.cache\kagglehub\datasets\pinxau1000\radioml2018\versions\2

 Volume in drive C is Windows SSD
 Volume Serial Number is 00BC-1571

 Directory of C:\Users\USER\.cache\kagglehub\datasets\pinxau1000\radioml2018\versions\2

06/03/2025  04:24 PM    <DIR>          .
06/03/2025  04:22 PM    <DIR>          ..
06/03/2025  04:24 PM               246 classes-fixed.json
06/03/2025  04:24 PM             1,632 classes-fixed.txt
06/03/2025  04:24 PM               262 classes.txt
06/03/2025  04:24 PM               172 datasets.desktop
06/03/2025  04:24 PM    21,449,148,312 GOLD_XYZ_OSC.0001_1024.hdf5
06/03/2025  04:24 PM            20,993 LICENSE.TXT
               6 File(s) 21,449,171,617 bytes
               2 Dir(s)  783,414,308,864 bytes free


In [12]:
dataset_path = "C:\\Users\\USER\\.cache\\kagglehub\\datasets\\pinxau1000\\radioml2018\\versions\\2\\GOLD_XYZ_OSC.0001_1024.hdf5"

In [26]:
!pip install timm

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   -------------------------- ------------- 1.6/2.4 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 9.0 MB/s eta 0:00:00

   ------------- -------------------------- 1/3 [huggingface_hub]
   ------------- -------------------------- 1/3 [huggingface_hub]
   ------------- -------------------------- 1/3 [huggingface_hub]
   ------------- -------------------------- 1/3 [huggingface_hub]
   ------------- -------------------------- 1/3 [huggingface_hub]
   ------------- -------------------------- 1/3 [huggingface_hub]
   ------------- -------------------------- 1/3 [huggingface_hub]
   -------------------------- ------------- 2/3 [timm]
   -------------------------- ------------- 2/3 [timm]
   -------------------------- ------------- 2/3 [timm]
   -------------------------- ------------- 2/3 [timm]
   -------------------------- ------------- 2/3 [timm]
   -------------------------- -----------

In [13]:
!pip install torch seaborn

In [14]:
!pip install --upgrade torch

In [15]:
!pip install torchaudio

In [16]:
!pip install torchvision

1. Load dataset dan preprocessing

In [17]:
!pip install torchsummary

In [18]:
from torchsummary import summary
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import torchaudio
from torchvision import transforms
import h5py

In [19]:
import h5py

# Misalkan kamu sudah memuat file HDF5
with h5py.File(dataset_path, 'r') as f:
    X = f['X'][:]  # Sinyal IQ
    Y = f['Y'][:]  # Label modulasi
    Z = f['Z'][:]  # Label SNR
    
    print(f"Shape of X (IQ data): {X.shape}")
    print(f"Shape of Y (Labels): {Y.shape}")
    print(f"Shape of Z (SNR): {Z.shape}")


Shape of X (IQ data): (2555904, 1024, 2)
Shape of Y (Labels): (2555904, 24)
Shape of Z (SNR): (2555904, 1)


2. Pendefinisian Class dan Objek

In [ ]:
# --- DEFINISI KELAS DATASET BARU ---

class SpectrogramDataset(Dataset):
    """
    Dataset ini mengambil sinyal IQ 1D, dan secara 'on-the-fly' mengubahnya
    menjadi gambar spectrogram 2D yang siap digunakan oleh model 2D.
    """
    def __init__(self, hdf5_path, classes, image_transform=None):
        with h5py.File(hdf5_path, 'r') as f:
            self.X = f['X']  # Data sinyal IQ mentah
            self.Y = f['Y']  # Label one-hot
        self.classes = classes
        self.image_transform = image_transform
        
        # Inisialisasi transformasi untuk membuat spectrogram.
        # n_fft adalah ukuran window untuk Fast Fourier Transform.
        self.spectrogram_generator = torchaudio.transforms.Spectrogram(n_fft=256,onesided=False)

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        # ===== DI SINILAH PROSES KONVERSI 1D -> 2D TERJADI =====

        # 1. Ambil sinyal I/Q untuk satu sampel. Shape: (1024, 2)
        iq_signal = self.X[idx]
        
        # 2. Gabungkan I dan Q menjadi sinyal kompleks.
        #    Operasi STFT (inti dari spectrogram) bekerja pada sinyal kompleks.
        complex_signal = torch.tensor(iq_signal[:, 0] + 1j * iq_signal[:, 1], dtype=torch.complex64)
        
        # 3. Buat Spectrogram dari sinyal kompleks.
        #    Hasilnya adalah tensor 2D yang merepresentasikan frekuensi vs. waktu.
        spectrogram = self.spectrogram_generator(complex_signal)
        
        # 4. Konversi ke Magnitudo dengan skala log.
        #    .abs() mengambil magnitudo, dan log1p menstabilkan rentang nilai
        #    agar lebih mudah dipelajari oleh model.
        log_spectrogram = torch.log1p(spectrogram.abs())
        
        # 5. Sesuaikan channel agar cocok dengan model 2D (misal: ResNet, EfficientNet).
        #    Model 2D pre-trained mengharapkan 3 channel (seperti gambar RGB).
        #    Kita duplikasi spectrogram (yang 1 channel) menjadi 3 channel.
        #    - .unsqueeze(0) menambah dimensi channel: (H, W) -> (1, H, W)
        #    - .repeat(3, 1, 1) mengulang channel tsb 3 kali: (1, H, W) -> (3, H, W)
        rgb_like_spectrogram = log_spectrogram.unsqueeze(0).repeat(3, 1, 1)

        # 6. Terapkan transformasi gambar (jika ada).
        #    Ini biasanya untuk me-resize gambar ke ukuran input model (misal 224x224)
        #    dan menormalisasi nilainya.
        if self.image_transform:
            rgb_like_spectrogram = self.image_transform(rgb_like_spectrogram)
        
        # 7. Ambil label dan konversi dari one-hot ke integer.
        label = torch.tensor(np.argmax(self.Y[idx]), dtype=torch.long)
        
        # 8. Kembalikan gambar spectrogram dan labelnya.
        return rgb_like_spectrogram, label

class BasicBlock(nn.Module):
     """ Basic Residual Block (used in ResNet) """
     def __init__(self, in_channels, out_channels):
         super(BasicBlock, self).__init__()
         self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
         self.bn1 = nn.BatchNorm1d(out_channels)
         self.relu = nn.ReLU()
         self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
         self.bn2 = nn.BatchNorm1d(out_channels)
         self.shortcut = nn.Sequential()
         if in_channels != out_channels:
             self.shortcut = nn.Sequential(
                 nn.Conv1d(in_channels, out_channels, kernel_size=1),
                 nn.BatchNorm1d(out_channels)
             )

     def forward(self, x):
         out = self.relu(self.bn1(self.conv1(x)))
         out = self.bn2(self.conv2(out))
         out += self.shortcut(x)  # residual connection
         out = self.relu(out)
         return out

class CNNResNetModel(nn.Module):
    def __init__(self, num_classes=8):
        super(CNNResNetModel, self).__init__()
        self.conv1 = nn.Conv1d(2, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        
        # ResNet blocks
        self.resnet_block1 = BasicBlock(64, 128)
        self.resnet_block2 = BasicBlock(128, 256)
        self.resnet_block3 = BasicBlock(256, 512)
        
        self.fc1 = nn.Linear(512 * 16, 1024)  # Adjust according to input length after pooling
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape from (batch_size, 1024, 2) to (batch_size, 2, 1024)
        # CNN part
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)  # shape becomes (batch_size, 64, 64)

        # ResNet part
        x = self.resnet_block1(x)  # (batch_size, 128, 64)
        x = self.pool(x)  # shape becomes (batch_size, 128, 32)
        
        x = self.resnet_block2(x)  # (batch_size, 256, 32)
        x = self.pool(x)  # shape becomes (batch_size, 256, 16)
        
        x = self.resnet_block3(x)  # (batch_size, 512, 16)
        x = self.pool(x)
        # Flattening the output for fully connected layers
        x = x.view(x.size(0), -1) # Flatten

        # Fully connected layers
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class RadioMLDataset(Dataset):
    """ Dataset for loading and preprocessing HDF5 file """
    def __init__(self, hdf5_path, classes):
        with h5py.File(hdf5_path, 'r') as f:
            self.X = f['X'][:]
            self.Y = f['Y'][:]
            self.SNR = f['Z'][:]
        self.classes = classes
        self.class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
        
    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        # Normalize IQ samples to range [-1, 1]
        iq_data = self.X[idx]  # normalize
        label = self.Y[idx]
        if isinstance(label, np.ndarray):
            label = np.argmax(label)
        iq_data_normalized = iq_data / np.max(np.abs(iq_data))
        iq_tensor = torch.FloatTensor(iq_data_normalized)
        label_tensor = torch.tensor(label, dtype=torch.long)
        return iq_tensor, label_tensor # torch.FloatTensor(iq_data), torch.tensor(label)

hdf5_path = 'C:\\Users\\USER\\.cache\\kagglehub\\datasets\\pinxau1000\\radioml2018\\versions\\2\\GOLD_XYZ_OSC.0001_1024.hdf5'
classes = ['BPSK', 'QPSK', '8PSK', 'QAM16', 'QAM64', 'CPFSK', 'GFSK', '4PAM']

dataset = RadioMLDataset(hdf5_path, classes)

# Melihat contoh data
print(f"Total samples: {len(dataset)}")
print(f"Shape of first sample (IQ): {dataset[0][0].shape}")
print(f"Label for first sample: {dataset[0][1]}")

Total samples: 2555904
Shape of first sample (IQ): torch.Size([1024, 2])
Label for first sample: 0


3. Subprogram Training dan Evaluasi

In [5]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    return running_loss / total, correct / total, all_labels, all_preds

4. Program Utama untuk Training dan Evaluasi

In [23]:
'''from torch.optim.lr_scheduler import ReduceLROnPlateau # Pastikan import ini ada

def main():
    # 1. Pengaturan Awal (Path dan Kelas)
    # Gunakan path lengkap dan benar yang sudah kita perbaiki
    hdf5_path = 'C:\\Users\\USER\\.cache\\kagglehub\\datasets\\pinxau1000\\radioml2018\\versions\\2\\GOLD_XYZ_OSC.0001_1024.hdf5'
    classes = ['BPSK', 'QPSK', '8PSK', 'QAM16', 'QAM64', 'CPFSK', 'GFSK', '4PAM']
    
    # Pastikan kelas CNNResNetModel sudah diperbarui dengan perbaikan `permute` dan `fc1`
    dataset = RadioMLDataset(hdf5_path, classes)

    # 2. Pembagian Data (80-20)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

    # 3. Inisialisasi Model, Optimizer, dan Scheduler
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Menggunakan device: {device}")
    
    model = CNNResNetModel(num_classes=len(classes)).to(device)
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer dengan weight_decay untuk regularisasi
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    
    # Scheduler untuk menyesuaikan learning rate (tanpa argumen 'verbose' agar kompatibel)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)

    # 4. Persiapan untuk Training Loop
    num_epochs = 25
    history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': []}
    best_test_acc = 0.0

    print("--- Memulai Training ---")
    for epoch in range(num_epochs):
        # Melatih model untuk satu epoch
        train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
        
        # Mengevaluasi model pada data testing
        test_loss, test_acc, _, _ = evaluate_model(model, test_loader, criterion, device)

        # Menyimpan riwayat loss dan akurasi
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        
        print(f"Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

        # Menyesuaikan learning rate berdasarkan akurasi testing
        scheduler.step(test_acc)
        
        # Menyimpan model terbaik
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            torch.save(model.state_dict(), 'best_model_weights.pth')
            print(f"    -> Model terbaik baru disimpan dengan akurasi: {best_test_acc:.4f}")

    print("--- Training Selesai ---")

    # 5. Visualisasi Hasil Training
    print("\nMembuat plot loss dan akurasi...")
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['test_loss'], label='Test Loss')
    plt.title('Grafik Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['test_acc'], label='Test Accuracy')
    plt.title('Grafik Akurasi per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

    # 6. Evaluasi Final dengan Model Terbaik
    print("\nMelakukan evaluasi akhir dengan model terbaik...")
    model.load_state_dict(torch.load('best_model_weights.pth'))
    
    _, _, true_labels, predicted_labels = evaluate_model(model, test_loader, criterion, device)
    
    # Menampilkan Confusion Matrix
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.title('Confusion Matrix dari Model Terbaik')
    plt.xlabel('Prediksi')
    plt.ylabel('Sebenarnya')
    plt.show()

    # Menampilkan Akurasi per Kelas
    print("\n--- Akurasi per Kelas ---")
    class_accuracy = cm.diagonal() / cm.sum(axis=1)
    for i, acc in enumerate(class_accuracy):
        print(f"  - Akurasi untuk {classes[i]:<7}: {acc:.4f}")


if __name__ == '__main__':
    main()'''

'from torch.optim.lr_scheduler import ReduceLROnPlateau # Pastikan import ini ada\n\ndef main():\n    # 1. Pengaturan Awal (Path dan Kelas)\n    # Gunakan path lengkap dan benar yang sudah kita perbaiki\n    hdf5_path = \'C:\\Users\\USER\\.cache\\kagglehub\\datasets\\pinxau1000\\radioml2018\\versions\\2\\GOLD_XYZ_OSC.0001_1024.hdf5\'\n    classes = [\'BPSK\', \'QPSK\', \'8PSK\', \'QAM16\', \'QAM64\', \'CPFSK\', \'GFSK\', \'4PAM\']\n    \n    # Pastikan kelas CNNResNetModel sudah diperbarui dengan perbaikan `permute` dan `fc1`\n    dataset = RadioMLDataset(hdf5_path, classes)\n\n    # 2. Pembagian Data (80-20)\n    train_size = int(0.8 * len(dataset))\n    test_size = len(dataset) - train_size\n    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])\n\n    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)\n    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

Membuat Spektrogram

In [ ]:
!pip install --upgrade ipywidgets

In [9]:
!pip install --upgrade torch

In [1]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.


In [ ]:
# Pastikan library ini sudah di-import di sel sebelumnya
import timm
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
import torchaudio
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim


def main_spectrogram():
    # 1. Pengaturan Awal (Path, Kelas, dan Transformasi)
    hdf5_path = 'C:\\Users\\USER\\.cache\\kagglehub\\datasets\\pinxau1000\\radioml2018\\versions\\2\\GOLD_XYZ_OSC.0001_1024.hdf5'
    classes = [
    'OOK','4ASK','8ASK','BPSK','QPSK','8PSK','16PSK','32PSK','64PSK',
    'QAM16','QAM32','QAM64','QAM128','QAM256','AM-SSB-WC','AM-SSB-SC',
    'AM-DSB-WC','AM-DSB-SC','FM','GMSK','OQPSK','WBFM','CPFSK','GFSK'
    ]

    # Buka file HDF5 untuk mendapatkan jumlah total sampel
    with h5py.File(hdf5_path, 'r') as f:
        total_samples = f['X'].shape[0]

    SUBSET_PERCENTAGE = 0.05  # 5% dari total sampel

    # Pilih 10% indeks secara acak
    subset_size = int(total_samples * SUBSET_PERCENTAGE)
    all_indices = np.arange(total_samples)
    subset_indices = np.random.choice(all_indices, size=subset_size, replace=False)
    
    print(f"Total sampel: {total_samples}")
    print(f"Mengambil {SUBSET_PERCENTAGE*100}% subset: {subset_size} sampel")

    # --- TAHAP 2: PRA-PEMROSESAN (MEMBUAT SPECTROGRAM UNTUK SUBSET) ---
    
    spectrograms_in_memory = []
    labels_in_memory = []
    
    spectrogram_generator = torchaudio.transforms.Spectrogram(n_fft=256, onesided=False)

    # Lakukan loop hanya pada indeks-indeks subset yang telah dipilih
    print("\nMemulai pra-pemrosesan untuk membuat spectrogram...")
    with h5py.File(hdf5_path, 'r') as f:
        for idx in tqdm(subset_indices, desc="Membuat Spectrogram"):
            iq_signal = f['X'][idx]
            label_onehot = f['Y'][idx]
            
            # Proses konversi sama seperti sebelumnya
            complex_signal = torch.tensor(iq_signal[:, 0] + 1j * iq_signal[:, 1], dtype=torch.complex64)
            spectrogram = spectrogram_generator(complex_signal)
            log_spectrogram = torch.log1p(spectrogram.abs())
            
            # Simpan spectrogram mentah (tanpa resize/normalize) ke memori
            spectrograms_in_memory.append(log_spectrogram)
            labels_in_memory.append(np.argmax(label_onehot))

    # --- TAHAP 3: MEMBUAT DATASET PYTORCH DARI DATA DI RAM ---

    # Ubah list menjadi tensor PyTorch tunggal
    all_spectrograms = torch.stack(spectrograms_in_memory)
    all_labels = torch.LongTensor(labels_in_memory)
    
    # Tambahkan dimensi channel (dari (N, H, W) -> (N, 1, H, W))
    all_spectrograms = all_spectrograms.unsqueeze(1)
    
    # Buat dataset PyTorch yang efisien dari tensor di memori
    # Kita tidak lagi menggunakan SpectrogramDataset, tapi TensorDataset
    precomputed_dataset = TensorDataset(all_spectrograms, all_labels)
    print("\nDataset dari spectrogram di memori berhasil dibuat.")
    print(f"Bentuk tensor spectrogram: {all_spectrograms.shape}")

    # --- TAHAP 4: TRAINING DAN EVALUASI (MENGGUNAKAN DATA SUBSET) ---

    # Definisikan transformasi (akan diterapkan oleh DataLoader)
    image_transforms = transforms.Compose([
        # Duplikasi channel agar menjadi 3 (agar cocok dengan model pre-trained)
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)), 
        transforms.Resize((224, 224)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Kelas wrapper untuk menerapkan transformasi
    class TransformDataset(Dataset):
        def __init__(self, dataset, transform):
            self.dataset = dataset
            self.transform = transform
        def __getitem__(self, index):
            x, y = self.dataset[index]
            return self.transform(x), y
        def __len__(self):
            return len(self.dataset)

    transformed_dataset = TransformDataset(precomputed_dataset, image_transforms)

    # Pembagian Data (80-20 dari 10% subset)
    train_size = int(0.8 * len(transformed_dataset))
    test_size = len(transformed_dataset) - train_size
    train_dataset, test_dataset = random_split(transformed_dataset, [train_size, test_size])
    print(f"Subset dibagi menjadi {train_size} training dan {test_size} testing.")
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

    # Inisialisasi Model, dll.
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Pelatihan akan menggunakan device: {device}")
    model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=len(classes)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
    
    # Loop Training
    num_epochs = 10
    print(f"\n--- Memulai Training pada {SUBSET_PERCENTAGE*100}% Data ---")
    # (Kode loop training dan evaluasi selanjutnya sama persis)
    for epoch in range(num_epochs):
        train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc, _, _ = evaluate_model(model, test_loader, criterion, device)
        print(f"Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
        scheduler.step(test_loss)

    # Evaluasi Akhir
    print("\n--- Evaluasi Akhir ---")
    _, _, true_labels, predicted_labels = evaluate_model(model, test_loader, criterion, device)
    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(16, 14))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.title(f'Confusion Matrix (Training pada {SUBSET_PERCENTAGE*100}% Data)')
    plt.show()

# Panggil fungsi utama yang baru
if __name__ == '__main__':
    main_spectrogram()

Total sampel: 2555904
Mengambil 5.0% subset: 127795 sampel

Memulai pra-pemrosesan untuk membuat spectrogram...


Membuat Spectrogram: 100%|██████████| 127795/127795 [01:00<00:00, 2117.32it/s]



Dataset dari spectrogram di memori berhasil dibuat.
Bentuk tensor spectrogram: torch.Size([127795, 1, 256, 9])
Subset dibagi menjadi 102236 training dan 25559 testing.

--- Memulai Training pada 5.0% Data ---


Evaluating: 100%|██████████| 400/400 [02:53<00:00,  2.31it/s]


Epoch [1/10] | Train Loss: 2.1625, Train Acc: 0.2713 | Test Loss: 2.0542, Test Acc: 0.2916


Training:  80%|████████  | 1282/1598 [35:01<08:38,  1.64s/it]


KeyboardInterrupt: 